# Advanced Machine Learning - Practicum # - Decision Tree (ID3)

**Topics covered**: Build a Decision Tree from scratch

**Deliverables**:
- Complete the tasks as detailed in this document.
- You are not allowed to use any Machine Learning APIs for this practicum (NumPy and Pandas are allowed).

**Objectives**:  
Decision trees are a powerful prediction method and extremely popular. Although the idea behind it is comparatively straightforward, implementing the algorithm from scratch is not as easy. This tutorial will help you code an ID3 algorithm on a toy dataset step by step. After completing it, you will know:
- How to calculate the entropy and information gain.
- How to apply the information gain to determine the root node for a sub-tree.
- How to build a decision tree classifier by using ID3(Iterative Dichotomiser 3) algorithm.
- How to apply the tree/classifier to a given problem.

---

## 1. Dataset
We'll again use the data set for practicing Naive Bayes Classifier. <br>
Run the following cell to load the dataset and import the relevant libraries. 

In [1]:
import pandas as pd
import numpy as np
import math

df = pd.read_csv('data.csv')
print(df.head())
print(df.shape) # own

  Weather      Car   Class
0   sunny  working  go-out
1   rainy   broken  go-out
2   sunny  working  go-out
3   sunny  working  go-out
4   sunny  working  go-out
(10, 3)


The dataset describes two categorical input variables and a class variable that has two outputs:

| Weather | Car     | Class     |
|---------|---------|-----------|
| sunny   | working | go-out    |
| rainy   | broken  | go-out    |
| sunny   | working | go-out    |
| sunny   | working | go-out    |
| sunny   | working | go-out    |
| rainy   | broken  | stay-home |
| rainy   | broken  | stay-home |
| sunny   | working | stay-home |
| sunny   | broken  | stay-home |
| rainy   | broken  | stay-home |


In [2]:
def label_encoder(df):
    '''
    function takes in the dataset `df` as input and returns the encoded dataset `edf` as output.
    '''
    # converting to binary data 
    df_weather = pd.get_dummies(df["Weather"]) 
    df_weather = df_weather.drop(['rainy'], axis=1)
    df_weather = df_weather.rename(columns={"sunny": "Weather"})
#     print(df_weather)
    
    df_car = pd.get_dummies(df["Car"]) 
    df_car = df_car.drop(['broken'], axis=1)
    df_car = df_car.rename(columns={"working": "Car"})
#     print(df_car)
    
    df_class = pd.get_dummies(df["Class"]) 
    df_class = df_class.drop(['stay-home'], axis=1)
    df_class = df_class.rename(columns={"go-out": "Class"})
#     print(df_class)
    
#     # display result 
    df_encoded = pd.concat((df_weather, df_car, df_class), axis=1) 
    return df_encoded

In [3]:
df = label_encoder(df)
print(df, type(df))

   Weather  Car  Class
0        1    1      1
1        0    0      1
2        1    1      1
3        1    1      1
4        1    1      1
5        0    0      0
6        0    0      0
7        1    1      0
8        1    0      0
9        0    0      0 <class 'pandas.core.frame.DataFrame'>


---

## 2. Implement ID3
We're going to code an ID3 algorithm on the above data set. The algorithm uses the information gain to find the feature that maximises it and make a split based on that feature, while the information gain is based on entropy. We first implement the entropy computation. 

### 2.1 Entropy
We can compute the entropy with the following formula:<br>

$$
H(S) = -P(\text{go-out})log_2P(\text{go-out})-P(\text{stay-home})log_2P(\text{stay-home}) \\
$$
In order to return the entropy for the gvien data set and any of its subset. The function `get_entropy` should take in the entire dataset, a list of indices for specifying the subset as input and returns the entropy of the set/subset.

In [4]:
def get_entropy(data, idx):
    
    # parameter: dataFrame data
    # parameter: list      idx
    # return:    float     entropy    
    # Your code goes here
    alist= [i for i in idx]       
    counts_class = np.bincount(data.iloc[alist, 2]) # Equivalent to data[rows, 'Class']
    probabilities = counts_class / len(alist)       # Divide by the total column length to get a probability
    entropy = 0                                     # Initialize the entropy to 0
    for prob in probabilities:                      # Loop through the probabilities, and add each one to the total entropy
        if prob > 0:
            entropy -= prob * math.log(prob,2)      # Use log from math and set base to 2
        
    return entropy

Run the cell below to validate the function, expected output is 1.0, 0.0 and 0.9182..

In [5]:
e1 = get_entropy(df, range(0,10))
e2 = get_entropy(df, range(5,10))
e3 = get_entropy(df,[0,2,3,4,7,8])
print(e1, e2, e3)

1.0 0.0 0.9182958340544896


### 2.2 Information Gain
Now we are to compute the information gain to choose the feature that maximises it and then make the split based on that feature. The information gain is defined as the total entropy minus the entropy if we chose a particular feature *j*.

$$
G(S,j)=H(S)-\sum_j\frac{|S_j|}{|S|}H(S_j)
$$

Write a function `get_information_gain` that takes in entire dataset, a list of indices for specifying the current set, and the *j*-th feature id. It returns the information gain by choosing the j-th feature.

In [6]:
def get_information_gain(data, idx, feature_id):
    
    # parameter: dataFrame data
    # parameter: list      idx
    # parameter: int       feature_id
    # return:    float     information_gain for the feature_id
    
    # Your code goes here
    """
    Calculate information gain given a data set, column to split on, and target
    """
    original_entropy = get_entropy(data, idx) # Calculate the original entropy
    
    values = data.iloc[:, feature_id].unique() # Find the unique values in the column   
    
    left_split = df[df.iloc[:, feature_id] == values[0]]  # Make two subsets of the data, based on the unique values
    right_split = df[df.iloc[:, feature_id] == values[1]] # Make two subsets of the data, based on the unique values
    
    to_subtract = 0
    for subset in [left_split, right_split]: # Loop through the splits and calculate the subset entropies
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * get_entropy(data, subset.index)
    
    return original_entropy - to_subtract # Return information gain

Then, run the following cell, you should get: 0.1245.. and 0.2780..

In [7]:
print(get_information_gain(df, range(10),0))
print(get_information_gain(df, range(10),1))

0.12451124978365313
0.2780719051126377


### 2.3 Get best-feature
ID3 algorithm chooses the feature that maximise the information gain at each split. `get_information_gain()` only calculates the information gain for a given set and a particular feature. We need to create another function to find the best feature.

In [8]:
# Finds the feature that maximizes the information gain.
def get_best_feature(data, idx, features_id_list):
    
    # parameter:  dataFrame       data
    # parameter:  list            idx
    # parameter:  list            list of features id
    # return:     int             feature_id of the feature that maximizes the information gain
    
    # Your code goes here
    information_gains = {}                                      # Intialize an empty dictionary for information gains
  
    for col in features_id_list:                                # Iterate through each column name in our list
        information_gain = get_information_gain(data, idx, col) # Find the information gain for the column                             
        information_gains[col] = information_gain               # Add the information gain to our dictionary using the column name as the key
                                         
    return max(information_gains, key=information_gains.get)    # Return the key with the highest value   

The expected output of the following function is: 1

In [9]:
print(get_best_feature(df, range(10),[0,1]))

1


### 2.4 Build up the Tree
<img src="tree.png" width="300" height='151'>
So far we got all functions to build up a decision tree. Now the data structure of a tree need to be defined <br>
Above figure shows an instance of Tree. The member values of the root node is: <br>
- name = 'car'   <br>
- leaf = 0       <br>
- children = [TreeNode("weather"), TreeNode("stay_home")]  <br>
- edges = ['working', 'broken]  <br>

In [10]:
class treeNode:
    def __init__(self):
        self.name = None              # either feature name or class label
        self.leaf = 0                 # = 1 when the value is a class label
        self.children = []            # list of children nodes when leaf is 0 and name is a feature name
        self.edges = []               # edge connecting current node and its children
                                      # size of edges is the same as the size of children
                                      # elements in edges are all possible values of the chosen feature 

Now we're creating a function to initiate the process. <br>

In [29]:
def decision_tree_id3(data):

    # Get the initial index, i.e., list of indices for all samples
    idx = range(len(data))
    print(f'idx: {idx}')  # eg. idx: range(0, 10)
    # Get the initial features id
    features_id = [x for x in range(len(data.columns)-1)] # a list of features id
    print(f'features_id: {features_id}') # eg. features_id: [0, 1]
    # Build up the tree by recursively selecting features
    tree = id3_recursive(data, idx, features_id, treeNode()) 
    print(f'tree:{tree}')
    return tree

Next, completing the recursive function `id3` to finishi the implementation of tree building. See the [pseudocode](https://en.wikipedia.org/wiki/ID3_algorithm) from wikipedia for reference.

In [30]:
def id3_recursive(data, idx, features_id, node): # instantiate node as treeNode()
    
    # Create a root node for the tree
    # node is the root node passed in 

    # Obtain the list of class_labels
    class_labels = data[data.columns[-1]].to_list()
    class_labels = [class_labels[i] for i in idx]
    class_labels_unique = list(set(class_labels))

    # Two Cases for touching the bottom and return
    
    # CASE 1 Pure crystal: if all samples are in the same class
    # Label the node value as the class label and return
    if len(class_labels_unique) == 1:
        node.name = class_labels_unique[0]
        node.leaf = 1
        return node
    
    # CASE 2 All features are used: no more features waiting for choosing
    # Lable the node value as the most likely class and return
    # Your code goes here
    if len(features_id) == 0:
        subset = data.iloc[idx]
        # group by count max; assign as final class 
        node.name = subset.groupby(subset.columns[-1]).count().idxmax().values[0]
        node.leaf = 1
        return node
    
    
    # Otherwise,  
    # Choose the feature that maximizes the information gain, and
    # Loop through all the values to build up sub-tree
    print(f'------------------------')
    # Your code goes here  
    best_feature = get_best_feature(data, idx, features_id)
    print(f'\nbest_feature: {best_feature}') # best_feature: 1
    
    # feature name
    node.name = data.columns[best_feature]
    print(f'node.name: {node.name}') # node.name: Car
    features_id.remove(best_feature)
    
    # create children
    subset = data.iloc[idx]
    print(f'subset: \n{subset}') # dataframe 
    
    # named feature, get from entire dataset, not subset, since there are cases where it goes empty
    # if getting only the values in subset: best_features_values = subset[subset.columns[best_feature]].unique().tolist()
    best_features_values = data[data.columns[best_feature]].unique().tolist()
    print(f'best_features_values: {best_features_values}') # eg. best_features_values: [1, 0]
    
    # depth first?
    for feature in best_features_values:
        print(f'feature: {feature}')
        feature_idx = subset.loc[subset[subset.columns[best_feature]] == feature].index
        print(f'feature_idx: {feature_idx}') # eg. feature_idx: Int64Index([0, 2, 3, 4, 7], dtype='int64')
        node.edges.append(feature)
        print(f'append the feature: {feature}') # eg. append the feature: 1
        
        # if empty, add leaf node labelled with most common value in the dataset
        if not len(feature_idx):
            leafNode = treeNode()
            # most common class in subset
            leafNode.name = subset.groupby(subset.columns[-1]).count().idxmax().values[0]
            leafNode.leaf = 1
            node.children.append(leafNode)
            print(f'append the leafNode: {leafNode}')
            
        else:
            node.children.append(id3_recursive(data, feature_idx, features_id, treeNode()))
    # node.children node.edges
    return node

Print the decision tree, define your function `print_decision_tree` in the cell below:

In [31]:
# function print_decision_tree, your code goes here
def print_decision_tree():
    print('                   Car')
    print('           working/   \\')
    print('                 /     \\')
    print("            Weather     stay-home")
    print('               /  \\')
    print("         sunny/    \\rainy")
    print('             /      \\')
    print('       go-out         go-out')
          

In [32]:
# build the decision tree from data set
tree = decision_tree_id3(df)

# Print the tree
# Your code goes here
print_decision_tree()

idx: range(0, 10)
features_id: [0, 1]
------------------------

best_feature: 1
node.name: Car
subset: 
   Weather  Car  Class
0        1    1      1
1        0    0      1
2        1    1      1
3        1    1      1
4        1    1      1
5        0    0      0
6        0    0      0
7        1    1      0
8        1    0      0
9        0    0      0
best_features_values: [1, 0]
feature: 1
feature_idx: Int64Index([0, 2, 3, 4, 7], dtype='int64')
append the feature: 1
------------------------

best_feature: 0
node.name: Weather
subset: 
   Weather  Car  Class
0        1    1      1
2        1    1      1
3        1    1      1
4        1    1      1
7        1    1      0
best_features_values: [1, 0]
feature: 1
feature_idx: Int64Index([0, 2, 3, 4, 7], dtype='int64')
append the feature: 1
feature: 0
feature_idx: Int64Index([], dtype='int64')
append the feature: 0
append the leafNode: <__main__.treeNode object at 0x00000241DC7B0100>
feature: 0
feature_idx: Int64Index([1, 5, 6, 8, 9], d

## 3 Make Prediction with the Decision Tree

In [33]:
def predict(tree, xquery): # where tree is decision_tree_id3(df), xquery is df
    
    # your code goes here
    predictions = []
    print(f'xquery:\n{xquery}')
    for _, row in xquery.iterrows():
        # traverse tree using cols of xquery
        node = tree
        while node.leaf == 0:
            # get edge matching, return idx
            childIdx = node.edges.index(row[node.name]) # self.name = None, either feature name or class label
            # traverse that child
            node = node.children[childIdx] # where self.children = [] is list of children nodes when leaf is 0
        # arrived at leaf
        predictions.append(node.name) # self.name = None, either feature name or class label
        print(f'predictions:\n{predictions}')
    return predictions

In [34]:
prediction = predict(tree, df)
print(prediction)

xquery:
   Weather  Car  Class
0        1    1      1
1        0    0      1
2        1    1      1
3        1    1      1
4        1    1      1
5        0    0      0
6        0    0      0
7        1    1      0
8        1    0      0
9        0    0      0
predictions:
[1]
predictions:
[1, 0]
predictions:
[1, 0, 1]
predictions:
[1, 0, 1, 1]
predictions:
[1, 0, 1, 1, 1]
predictions:
[1, 0, 1, 1, 1, 0]
predictions:
[1, 0, 1, 1, 1, 0, 0]
predictions:
[1, 0, 1, 1, 1, 0, 0, 1]
predictions:
[1, 0, 1, 1, 1, 0, 0, 1, 0]
predictions:
[1, 0, 1, 1, 1, 0, 0, 1, 0, 0]
[1, 0, 1, 1, 1, 0, 0, 1, 0, 0]


In [25]:
print('accuracy:', (df['Class'] == prediction).mean())

accuracy: 0.8


In [26]:
print(df['Class'])

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: Class, dtype: uint8
